## Json 파일 읽어드리기

TLC Trip Record Data 출처: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [40]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local")\
.appName("spark-sql").getOrCreate()

In [4]:
df = spark.read.format('json')\
.load("learning_spark_data/2015-summary.json")

In [5]:
df.count()

256

In [7]:
df.dtypes

[('DEST_COUNTRY_NAME', 'string'),
 ('ORIGIN_COUNTRY_NAME', 'string'),
 ('count', 'bigint')]

In [9]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [11]:
df.take(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [13]:
df.select('count').show(2) # 데이터의  형식그대로 보고 싶을 경우

+-----+
|count|
+-----+
|   15|
|    1|
+-----+
only showing top 2 rows



In [14]:
df.select('DEST_COUNTRY_NAME').show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [17]:
df.select('DEST_COUNTRY_NAME').distinct().show() # 컬럼의 요소를 보는 방법

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|            Anguilla|
|              Russia|
|            Paraguay|
|             Senegal|
|              Sweden|
|            Kiribati|
|              Guyana|
|         Philippines|
|            Djibouti|
|            Malaysia|
|           Singapore|
|                Fiji|
|              Turkey|
|                Iraq|
|             Germany|
|              Jordan|
|               Palau|
|Turks and Caicos ...|
|              France|
|              Greece|
+--------------------+
only showing top 20 rows



In [18]:
df.select('DEST_COUNTRY_NAME').distinct().cache()

DataFrame[DEST_COUNTRY_NAME: string]

**cache**: 한 번 계산한 결과를 메모리에 저장해두고 다음부터는 빠르게 가져올 수 있다.

In [20]:
# Row class를 이용한 단일 레코드 생성

from pyspark.sql import Row
myRow = Row('hello', None, 1, False)
myRow

<Row('hello', None, 1, False)>

In [23]:
# 새로운 컬럼 추가
from pyspark.sql.functions import expr

df3 = df.withColumn('withinCountry',expr('ORIGIN_COUNTRY_NAME== DEST_COUNTRY_NAME'))
df3

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, withinCountry: boolean]

In [24]:
df3.show(3)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
+-----------------+-------------------+-----+-------------+
only showing top 3 rows



In [27]:
df3.filter(df3.withinCountry == True).show()

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [33]:
from pyspark.sql.functions import expr

df4 = df3.withColumn(
    'Category',
    expr("CASE WHEN count >= 10 THEN 'upper' ELSE 'under' END")
)

df4.show(5)

+-----------------+-------------------+-----+-------------+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|Category|
+-----------------+-------------------+-----+-------------+--------+
|    United States|            Romania|   15|        false|   upper|
|    United States|            Croatia|    1|        false|   under|
|    United States|            Ireland|  344|        false|   upper|
|            Egypt|      United States|   15|        false|   upper|
|    United States|              India|   62|        false|   upper|
+-----------------+-------------------+-----+-------------+--------+
only showing top 5 rows



DataFrame 의 select(), where(), filter() - Transformation

             show(), count() - Action

In [41]:
spark.stop()

## csv 파일 읽어들이기

In [42]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ReadCSV").getOrCreate()

# CSV 파일 읽기
dept_df = spark.read.csv("learning_spark_data/dept.csv", header=True, inferSchema=True)
dept_df.show()

+------+----------+--------+
|deptno|     dname|     loc|
+------+----------+--------+
|    10|ACCOUNTING|NEW YORK|
|    20|  RESEARCH|  DALLAS|
|    30|     SALES| CHICAGO|
|    40|OPERATIONS|  BOSTON|
+------+----------+--------+



In [45]:
dept_df.printSchema()

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [47]:
dept_df.count()

4

In [50]:
emp_df = spark.read.csv("learning_spark_data/emp.csv", header=True, inferSchema=True)
emp_df.show(5)

+-----+------+--------+----+----------+----+----+------+
|empno| ename|     job| mgr|  hiredate| sal|comm|deptno|
+-----+------+--------+----+----------+----+----+------+
| 7369| SMITH|   CLERK|7902|1980-12-17| 800|NULL|    20|
| 7499| ALLEN|SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD|SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES| MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7654|MARTIN|SALESMAN|7698|1981-09-28|1250|1400|    30|
+-----+------+--------+----+----------+----+----+------+
only showing top 5 rows



In [46]:
emp_df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: date (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)



In [48]:
emp_df.count()

15

In [54]:
# 컬럼명은 대소문자를 구분하지 않는다.
emp_df.select('ename','deptno').show()

+------+------+
| ename|deptno|
+------+------+
| SMITH|    20|
| ALLEN|    30|
|  WARD|    30|
| JONES|    20|
|MARTIN|    30|
| BLAKE|    30|
| CLARK|    10|
| SCOTT|    20|
|  KING|    10|
|TURNER|    30|
| ADAMS|    20|
| JAMES|    30|
|  FORD|    20|
|MILLER|    10|
|  JACK|    70|
+------+------+



In [55]:
emp_df.select('*').where('deptno=20').show()

+-----+-----+-------+----+----------+----+----+------+
|empno|ename|    job| mgr|  hiredate| sal|comm|deptno|
+-----+-----+-------+----+----------+----+----+------+
| 7369|SMITH|  CLERK|7902|1980-12-17| 800|NULL|    20|
| 7566|JONES|MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7788|SCOTT|ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7876|ADAMS|  CLERK|7788|1987-05-23|1100|NULL|    20|
| 7902| FORD|ANALYST|7566|1981-12-03|3000|NULL|    20|
+-----+-----+-------+----+----------+----+----+------+



In [57]:
emp_df.selectExpr('count(*)').show()

+--------+
|count(1)|
+--------+
|      15|
+--------+



In [60]:
from pyspark.sql.functions import countDistinct
emp_df.select(countDistinct('job')).show()

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [61]:
from pyspark.sql.functions import approx_count_distinct
emp_df.select(approx_count_distinct('job',0.1)).show()

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



**approx_count_distinct**
| 파라미터                                | 설명                                             |
| ----------------------------------- | ---------------------------------------------- |
| `column_name`                       | 고유값 개수를 추정할 컬럼 이름                              |
| `rsd` (Relative Standard Deviation) | 허용 오차 범위. 작을수록 정확하지만 메모리와 시간이 더 듦. 기본값은 `0.05` |

In [66]:
from pyspark.sql.functions import first, last, min, max, sum, avg

emp_df.select('ename','sal').show()

+------+----+
| ename| sal|
+------+----+
| SMITH| 800|
| ALLEN|1600|
|  WARD|1250|
| JONES|2975|
|MARTIN|1250|
| BLAKE|2850|
| CLARK|2450|
| SCOTT|3000|
|  KING|5000|
|TURNER|1500|
| ADAMS|1100|
| JAMES| 950|
|  FORD|3000|
|MILLER|1300|
|  JACK|3200|
+------+----+



In [67]:
emp_df.select(
    first("sal").alias("first_salary"),
    last("sal").alias("last_salary"),
    min("sal").alias("min_salary"),
    max("sal").alias("max_salary"),
    sum("sal").alias("sum_salary"),
    avg("sal").alias("avg_salary")
).show()

+------------+-----------+----------+----------+----------+------------------+
|first_salary|last_salary|min_salary|max_salary|sum_salary|        avg_salary|
+------------+-----------+----------+----------+----------+------------------+
|         800|       3200|       800|      5000|     32225|2148.3333333333335|
+------------+-----------+----------+----------+----------+------------------+



transation vs. avg vs. mean

In [75]:
# 거래 건수
total_count = emp_df.selectExpr("count(*) as total_transaction").collect()[0]["total_transaction"]

# 거래 총액
total_sum = emp_df.selectExpr("sum(sal) as total_transaction").collect()[0]["total_transaction"]

# 평균 계산
total_mean = total_sum / total_count
print(f"평균 거래 금액: {total_mean}")

평균 거래 금액: 2148.3333333333335


In [77]:
from pyspark.sql.functions import avg, format_number

# 소숫점 두자리 까지 나오도록 
emp_df.select(format_number(avg("sal"), 2).alias("avg_salary")).show()

+----------+
|avg_salary|
+----------+
|  2,148.33|
+----------+



In [72]:
from pyspark.sql.functions import mean

emp_df.select(mean("sal").alias("mean_salary")).show()

+------------------+
|       mean_salary|
+------------------+
|2148.3333333333335|
+------------------+



In [78]:
# 그룹화
emp_df.groupby('job').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



In [81]:
from pyspark.sql.functions import count, sum
emp_df.groupby('job').agg(
    count('job').alias('qty'),
    expr('count(job)').alias('job_count'),
    sum('sal').alias('total_sal')
).show()

+---------+---+---------+---------+
|      job|qty|job_count|total_sal|
+---------+---+---------+---------+
|  ANALYST|  2|        2|     6000|
| SALESMAN|  4|        4|     5600|
|    CLERK|  5|        5|     7350|
|  MANAGER|  3|        3|     8275|
|PRESIDENT|  1|        1|     5000|
+---------+---+---------+---------+



In [84]:
from pyspark.sql.functions import stddev
# sal의 평균과 표준편차 계산
emp_df.groupby('job').agg(
    format_number(avg('sal'), 2).alias('SAL_AVG'),
    format_number(stddev('sal'), 2).alias('SAL_STDEV')
).show()

+---------+--------+---------+
|      job| SAL_AVG|SAL_STDEV|
+---------+--------+---------+
|  ANALYST|3,000.00|     0.00|
| SALESMAN|1,400.00|   177.95|
|    CLERK|1,470.00|   984.63|
|  MANAGER|2,758.33|   274.24|
|PRESIDENT|5,000.00|     NULL|
+---------+--------+---------+



In [86]:
# 급여 Top10 
emp_df.select('ename','sal') \
      .orderBy('sal', ascending=False) \
      .limit(10) \
      .show()

+------+----+
| ename| sal|
+------+----+
|  KING|5000|
|  JACK|3200|
| SCOTT|3000|
|  FORD|3000|
| JONES|2975|
| BLAKE|2850|
| CLARK|2450|
| ALLEN|1600|
|TURNER|1500|
|MILLER|1300|
+------+----+



In [94]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc
# 💡 Window 정의
salary_window = Window.orderBy(desc("sal"))

# 💡 랭크 추가
ranked_df = emp_df.withColumn("salary_rank", rank().over(salary_window))

# 💡 결과 보기
ranked_df.show(10)

+-----+------+---------+----+----------+----+----+------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|salary_rank|
+-----+------+---------+----+----------+----+----+------+-----------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|          1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|          2|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|          3|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|          3|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|          5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|          6|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|          7|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|          8|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|          9|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|         10|
+-----+------+---------+----+----------+----+----+------+-----------+
only showing top 10 

In [96]:
from pyspark.sql.functions import rank, dense_rank, row_number, desc
# WindowSpec: 직무(job)별로 sal 내림차순 정렬
job_window = Window.partitionBy("job").orderBy(desc("sal"))

# 직무별 순위 컬럼 추가
emp_df.withColumn("rank", rank().over(job_window)) \
      .withColumn("dense_rank", dense_rank().over(job_window)) \
      .withColumn("row_number", row_number().over(job_window)) \
      .select("ename", "job", "sal", "rank", "dense_rank", "row_number") \
      .show()

+------+---------+----+----+----------+----------+
| ename|      job| sal|rank|dense_rank|row_number|
+------+---------+----+----+----------+----------+
| SCOTT|  ANALYST|3000|   1|         1|         1|
|  FORD|  ANALYST|3000|   1|         1|         2|
|  JACK|    CLERK|3200|   1|         1|         1|
|MILLER|    CLERK|1300|   2|         2|         2|
| ADAMS|    CLERK|1100|   3|         3|         3|
| JAMES|    CLERK| 950|   4|         4|         4|
| SMITH|    CLERK| 800|   5|         5|         5|
| JONES|  MANAGER|2975|   1|         1|         1|
| BLAKE|  MANAGER|2850|   2|         2|         2|
| CLARK|  MANAGER|2450|   3|         3|         3|
|  KING|PRESIDENT|5000|   1|         1|         1|
| ALLEN| SALESMAN|1600|   1|         1|         1|
|TURNER| SALESMAN|1500|   2|         2|         2|
|  WARD| SALESMAN|1250|   3|         3|         3|
|MARTIN| SALESMAN|1250|   3|         3|         4|
+------+---------+----+----+----------+----------+



### 부서별  직업별 소계

In [102]:
joined_df = emp_df.join(dept_df, on="deptno", how="inner")
joined_df.show()

+------+-----+------+---------+----+----------+----+----+----------+--------+
|deptno|empno| ename|      job| mgr|  hiredate| sal|comm|     dname|     loc|
+------+-----+------+---------+----+----------+----+----+----------+--------+
|    20| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|  RESEARCH|  DALLAS|
|    30| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|     SALES| CHICAGO|
|    30| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|     SALES| CHICAGO|
|    20| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|  RESEARCH|  DALLAS|
|    30| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|     SALES| CHICAGO|
|    30| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|     SALES| CHICAGO|
|    10| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|ACCOUNTING|NEW YORK|
|    20| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|  RESEARCH|  DALLAS|
|    10| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|ACCOUNTING|NEW YORK|
|    30| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|     SA

In [106]:
# 윈도우의 정의: 부서 + 직업별 파티션
window_spe = Window.partitionBy('deptno','job')

# 윈도우 함수로 소계구하기
joined_df.withColumn('job_deptno_total_sal',sum('sal').over(window_spe))\
.select('ename','deptno','job','sal','job_deptno_total_sal').show()

+------+------+---------+----+--------------------+
| ename|deptno|      job| sal|job_deptno_total_sal|
+------+------+---------+----+--------------------+
|MILLER|    10|    CLERK|1300|                1300|
| CLARK|    10|  MANAGER|2450|                2450|
|  KING|    10|PRESIDENT|5000|                5000|
| SCOTT|    20|  ANALYST|3000|                6000|
|  FORD|    20|  ANALYST|3000|                6000|
| SMITH|    20|    CLERK| 800|                1900|
| ADAMS|    20|    CLERK|1100|                1900|
| JONES|    20|  MANAGER|2975|                2975|
| JAMES|    30|    CLERK| 950|                 950|
| BLAKE|    30|  MANAGER|2850|                2850|
| ALLEN|    30| SALESMAN|1600|                5600|
|  WARD|    30| SALESMAN|1250|                5600|
|MARTIN|    30| SALESMAN|1250|                5600|
|TURNER|    30| SALESMAN|1500|                5600|
+------+------+---------+----+--------------------+



In [114]:
# 부서별 급여 합계 구하기
df = joined_df.groupby('deptno')\
.agg(sum('sal').alias('deptno_sal'))

# 윈도우 정의: 급여 합계 기준 내림차순 정렬
detpno_rank = Window.orderBy(df['deptno_sal'].desc())

# 순위칼럼추가
final_df = df.withColumn("dept_rank", rank().over(detpno_rank)).show()

+------+----------+---------+
|deptno|deptno_sal|dept_rank|
+------+----------+---------+
|    20|     10875|        1|
|    30|      9400|        2|
|    10|      8750|        3|
+------+----------+---------+



In [115]:
# cube-> 부서(deptno), 직무(job)별 급여 합계와 전체 집계 모두 계산
df_cube = joined_df.cube("deptno", "job").agg(sum("sal").alias("total_sal"))

df_cube.show()

+------+---------+---------+
|deptno|      job|total_sal|
+------+---------+---------+
|  NULL|PRESIDENT|     5000|
|    10|    CLERK|     1300|
|    20|  ANALYST|     6000|
|  NULL|  ANALYST|     6000|
|    30| SALESMAN|     5600|
|    10|     NULL|     8750|
|  NULL| SALESMAN|     5600|
|  NULL|    CLERK|     4150|
|    30|  MANAGER|     2850|
|  NULL|     NULL|    29025|
|    10|PRESIDENT|     5000|
|  NULL|  MANAGER|     8275|
|    30|     NULL|     9400|
|    10|  MANAGER|     2450|
|    20|     NULL|    10875|
|    20|  MANAGER|     2975|
|    20|    CLERK|     1900|
|    30|    CLERK|      950|
+------+---------+---------+



[참고]

- rollup도 마찬가지로 정렬 안 하면 출력 순서가 뒤죽박죽일 수 있습니다.

- 실제 분석, 리포트용으로는 항상 orderBy를 쓰는 게 관례입니다.

In [117]:
df_cube.orderBy('deptno', 'job').show()

+------+---------+---------+
|deptno|      job|total_sal|
+------+---------+---------+
|  NULL|     NULL|    29025|
|  NULL|  ANALYST|     6000|
|  NULL|    CLERK|     4150|
|  NULL|  MANAGER|     8275|
|  NULL|PRESIDENT|     5000|
|  NULL| SALESMAN|     5600|
|    10|     NULL|     8750|
|    10|    CLERK|     1300|
|    10|  MANAGER|     2450|
|    10|PRESIDENT|     5000|
|    20|     NULL|    10875|
|    20|  ANALYST|     6000|
|    20|    CLERK|     1900|
|    20|  MANAGER|     2975|
|    30|     NULL|     9400|
|    30|    CLERK|      950|
|    30|  MANAGER|     2850|
|    30| SALESMAN|     5600|
+------+---------+---------+



In [116]:
df_cube.orderBy('deptno', 'job', ascending=[True, True]).show()

+------+---------+---------+
|deptno|      job|total_sal|
+------+---------+---------+
|  NULL|     NULL|    29025|
|  NULL|  ANALYST|     6000|
|  NULL|    CLERK|     4150|
|  NULL|  MANAGER|     8275|
|  NULL|PRESIDENT|     5000|
|  NULL| SALESMAN|     5600|
|    10|     NULL|     8750|
|    10|    CLERK|     1300|
|    10|  MANAGER|     2450|
|    10|PRESIDENT|     5000|
|    20|     NULL|    10875|
|    20|  ANALYST|     6000|
|    20|    CLERK|     1900|
|    20|  MANAGER|     2975|
|    30|     NULL|     9400|
|    30|    CLERK|      950|
|    30|  MANAGER|     2850|
|    30| SALESMAN|     5600|
+------+---------+---------+



In [118]:
emp_df.cube('deptno','job').agg(count('*'), avg('sal'))\
    .orderBy('deptno','job').show()

+------+---------+--------+------------------+
|deptno|      job|count(1)|          avg(sal)|
+------+---------+--------+------------------+
|  NULL|     NULL|      15|2148.3333333333335|
|  NULL|  ANALYST|       2|            3000.0|
|  NULL|    CLERK|       5|            1470.0|
|  NULL|  MANAGER|       3|2758.3333333333335|
|  NULL|PRESIDENT|       1|            5000.0|
|  NULL| SALESMAN|       4|            1400.0|
|    10|     NULL|       3|2916.6666666666665|
|    10|    CLERK|       1|            1300.0|
|    10|  MANAGER|       1|            2450.0|
|    10|PRESIDENT|       1|            5000.0|
|    20|     NULL|       5|            2175.0|
|    20|  ANALYST|       2|            3000.0|
|    20|    CLERK|       2|             950.0|
|    20|  MANAGER|       1|            2975.0|
|    30|     NULL|       6|1566.6666666666667|
|    30|    CLERK|       1|             950.0|
|    30|  MANAGER|       1|            2850.0|
|    30| SALESMAN|       4|            1400.0|
|    70|     

In [120]:
spark.stop()